In [1]:
import os
import torch
import copy
import random
import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.metrics import r2_score, mean_squared_error

def fix_random(seed: int) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)

    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # slower

fix_random(42)

if not os.path.exists('tabularML'):
    os.makedirs('tabularML')

In [2]:
df = pd.read_csv('dataset.csv')

# PyTorch Device
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print('Using device:', device)

Using device: cuda


In [3]:
#print only the year column
df.drop(['year','rating_count'], axis=1, inplace=True)

In [4]:
X = df.drop('rating', axis=1)
Y = df['rating']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=42)

X_train = X_train.to_numpy()
X_val = X_val.to_numpy()
X_test = X_test.to_numpy()
Y_train = Y_train.to_numpy()
Y_val = Y_val.to_numpy()
Y_test = Y_test.to_numpy()

Y_train = Y_train.reshape(-1, 1)
Y_val = Y_val.reshape(-1, 1)
print(f'Number of training samples: {X_train.shape[0]}')
print(f'Number of validation samples: {X_val.shape[0]}')
print(f'Number of testing samples: {X_test.shape[0]}')
print(f'\nNumber of features: {X_train.shape[1]}')

Number of training samples: 9934
Number of validation samples: 1104
Number of testing samples: 2760

Number of features: 1148


In [5]:
# Hyperparameters
nums_epochs = [100]
batch_sizes = [64,128]
patience = [10]
n_d_a = [8,16,32]
n_shared = [2,3]
n_indipendent = [2,3]
n_step = [3,4,5,6,7]
gamma = [1.3]
epsilon = [1e-8]

hyperparameters = itertools.product(n_d_a, n_step,n_indipendent,n_shared, gamma, epsilon,nums_epochs, batch_sizes)
n_comb = len(n_d_a)*len(n_step)*len(n_indipendent)*len(n_shared)*len(gamma)*len(epsilon)*len(nums_epochs)*len(batch_sizes)
print (f'Number of hyperparameter combinations: {n_comb}')

Number of hyperparameter combinations: 120


In [6]:
def get_model(n_d_a, n_step,n_indipendent,n_shared, gamma, epsilon):
    model = TabNetRegressor(
        n_d=n_d_a,
        n_a=n_d_a,
        n_steps=n_step,
        gamma=gamma,
        optimizer_fn=torch.optim.Adam,
        n_independent=n_indipendent,
        n_shared=n_shared,
        epsilon=epsilon,
        seed=42    
    )
    return model

In [7]:
if os.path.exists('tabularML/training'):
    os.system('rm -r tabularML/training')
else:
    os.makedirs('tabularML/training')

current_iter = 0

best_mse = float('inf')
best_model = None
best_n_d = None
best_n_a = None
best_n_step = None
best_n_indipendent = None
best_n_shared = None
best_gamma = None
best_batch_size = None

for n_d_a, n_step,n_indipendent,n_shared, gamma, epsilon,nums_epochs, batch_sizes in hyperparameters:
    current_iter += 1

    print("\nIterations {}/{} - Hyperparameters:  batch_sizes={}, nums_epochs={}, n_d={}, n_a={}, n_step={}, n_indipendent={}, n_shared={}, gamma={}, epsilon={}".format(
        current_iter, n_comb, batch_sizes, nums_epochs, n_d_a, n_d_a, n_step, n_indipendent, n_shared, gamma, epsilon ))

    model = get_model(n_d_a, n_step, n_indipendent, n_shared, gamma, epsilon)
    
    log_name = "batch_size="+str(batch_sizes)+"n_d="+str(n_d_a)+"n_a="+str(n_d_a)+"n_step="+str(n_step)+"n_indipendent="+str(n_indipendent)+"n_shared="+str(n_shared)+"gamma="+str(gamma)+"epsilon="+str(epsilon)
    
    # start tensorboard
    writer = SummaryWriter('run/tabularML/training/'+log_name)
    
    # train
    model.fit(
                X_train=X_train,
                y_train=Y_train,
                eval_set=[(X_val, Y_val)],
                eval_metric=['mse'],
                patience=10,
                batch_size=batch_sizes,
                virtual_batch_size=128,
                num_workers=0,
                drop_last=False,
                max_epochs=nums_epochs,
            )
    

    preds = model.predict(X_test)
    mse = mean_squared_error(Y_test, preds)
    
    writer.add_hparams({'n_d':n_d_a, 'n_a':n_d_a, 'n_step':n_step, 'n_indipendent':n_indipendent, 'n_shared':n_shared, 'gamma':gamma, 'epsilon':epsilon, 'batch_sizes':batch_sizes, 'nums_epochs':nums_epochs }, {'hparam/mse': mse})

    if mse < best_mse:
        best_mse = mse
        best_n_d = n_d_a
        best_n_a = n_d_a
        best_n_step = n_step
        best_n_indipendent = n_indipendent
        best_n_shared = n_shared
        best_gamma = gamma
        best_batch_size = batch_sizes
        best_model = copy.deepcopy(model) 
        
    writer.flush()            
            
    print("Model MSE: {:.6f} - Best MSE: {:.6f}".format(mse, best_mse))
    print("Model R2 Score: {:.6f} - Best R2 Score: {:.6f}".format(r2_score(Y_test, preds), r2_score(Y_test, best_model.predict(X_test))))



Iterations 1/120 - Hyperparameters:  batch_sizes=64, nums_epochs=100, n_d=8, n_a=8, n_step=3, n_indipendent=2, n_shared=2, gamma=1.3, epsilon=1e-08


C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.82736 | val_0_mse: 0.22076 |  0:00:10s
epoch 1  | loss: 0.11293 | val_0_mse: 0.11419 |  0:00:17s
epoch 2  | loss: 0.08793 | val_0_mse: 0.05898 |  0:00:24s
epoch 3  | loss: 0.06261 | val_0_mse: 0.04983 |  0:00:30s
epoch 4  | loss: 0.04976 | val_0_mse: 0.03972 |  0:00:37s
epoch 5  | loss: 0.03588 | val_0_mse: 0.02088 |  0:00:43s
epoch 6  | loss: 0.02648 | val_0_mse: 0.01735 |  0:00:49s
epoch 7  | loss: 0.02457 | val_0_mse: 0.03767 |  0:00:54s
epoch 8  | loss: 0.02418 | val_0_mse: 0.01295 |  0:01:00s
epoch 9  | loss: 0.021   | val_0_mse: 0.01947 |  0:01:05s
epoch 10 | loss: 0.02029 | val_0_mse: 0.03073 |  0:01:11s
epoch 11 | loss: 0.01729 | val_0_mse: 0.00913 |  0:01:16s
epoch 12 | loss: 0.01894 | val_0_mse: 0.0241  |  0:01:22s
epoch 13 | loss: 0.02272 | val_0_mse: 0.03062 |  0:01:27s
epoch 14 | loss: 0.0156  | val_0_mse: 0.01045 |  0:01:33s
epoch 15 | loss: 0.01981 | val_0_mse: 0.00983 |  0:01:38s
epoch 16 | loss: 0.02114 | val_0_mse: 0.01652 |  0:01:44s
epoch 17 | los

C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Model MSE: 0.006265 - Best MSE: 0.006265
Model R2 Score: 0.972224 - Best R2 Score: 0.972224

Iterations 2/120 - Hyperparameters:  batch_sizes=128, nums_epochs=100, n_d=8, n_a=8, n_step=3, n_indipendent=2, n_shared=2, gamma=1.3, epsilon=1e-08


...

Model MSE: 0.009777 - Best MSE: 0.004804
Model R2 Score: 0.956655 - Best R2 Score: 0.978703

Iterations 120/120 - Hyperparameters:  batch_sizes=128, nums_epochs=100, n_d=32, n_a=32, n_step=7, n_indipendent=3, n_shared=3, gamma=1.3, epsilon=1e-08


C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_tabnet\abstract_model.py:75: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.04621 | val_0_mse: 0.89179 |  0:00:06s
epoch 1  | loss: 0.65459 | val_0_mse: 0.55104 |  0:00:13s
epoch 2  | loss: 0.27058 | val_0_mse: 0.21246 |  0:00:20s
epoch 3  | loss: 0.14727 | val_0_mse: 0.17287 |  0:00:27s
epoch 4  | loss: 0.1315  | val_0_mse: 0.11316 |  0:00:34s
epoch 5  | loss: 0.09004 | val_0_mse: 0.09714 |  0:00:41s
epoch 6  | loss: 0.08564 | val_0_mse: 0.0746  |  0:00:48s
epoch 7  | loss: 0.08673 | val_0_mse: 0.09315 |  0:00:55s
epoch 8  | loss: 0.07454 | val_0_mse: 0.07321 |  0:01:01s
epoch 9  | loss: 0.08269 | val_0_mse: 0.07691 |  0:01:08s
epoch 10 | loss: 0.08133 | val_0_mse: 0.10324 |  0:01:15s
epoch 11 | loss: 0.07636 | val_0_mse: 0.06642 |  0:01:22s
epoch 12 | loss: 0.0662  | val_0_mse: 0.06662 |  0:01:29s
epoch 13 | loss: 0.05649 | val_0_mse: 0.04664 |  0:01:36s
epoch 14 | loss: 0.04468 | val_0_mse: 0.04308 |  0:01:42s
epoch 15 | loss: 0.03865 | val_0_mse: 0.04028 |  0:01:49s
epoch 16 | loss: 0.0495  | val_0_mse: 0.0324  |  0:01:56s
epoch 17 | los

C:\Users\franc\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pytorch_tabnet\callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Model MSE: 0.006136 - Best MSE: 0.004804
Model R2 Score: 0.972795 - Best R2 Score: 0.978703


In [11]:

print(best_n_d)
print(best_n_a)
print(best_n_step)
print(best_n_indipendent)
print(best_n_shared)
print(best_gamma)
print(best_batch_size)

8
8
4
2
2
1.3
128
